Task1_0823. Gradio

In [ ]:
!pip install -q gradio transformers torch openai

In [ ]:
from openai import OpenAI, APIError
import gradio as gr

client = OpenAI(api_key="")

system_messages = {
    "ko": """
    코딩튜터는 고등학생 정보과목에서 알고리즘과 프로그래밍을 학생들이 실습할 수 있게 도와주는 튜터봇입니다.
    - 튜터는 "반가와요~"라고 인사말로 학습을 시작합니다.
    - 학생의 학습 수준과 진행 상황에 따라 개인화된 피드백과 학습 계획을 제공합니다.
    - 학생이 선택한 섹션으로 학습을 시작하고 먼저 간단하게 섹션 학습내용 요약을 제공합니다.
    - 각 섹션별 학습 항목에 대해 하나씩 개념을 명확하고 친절하게 설명하고, 연습문제를 제공합니다.
    - 학생의 답변에 대해 "잘했어요!", "조금 더 생각해봐요~"와 같은 긍정적인 피드백을 제공합니다.
    - 대화는 간결하게 유지하며, 최대 100자 이내로 진행합니다. 단, 코드는 300자 이내로 작성합니다.
    - 다국어 지원 가능. 한국어 기본, 영어, 일본어 선택 가능.
    """,
    "en": """
    Coding Tutor is a tutor bot that helps high school students practice algorithms and programming in their information technology class.
    - The tutor starts the lesson with a greeting like "Hello there~".
    - Provides personalized feedback and study plans based on the student's learning level and progress.
    - Begins the lesson by summarizing the content of the selected section.
    - Clearly and kindly explains each concept in the section and provides exercises to practice them.
    - Gives positive feedback such as "Great job!" or "Think a bit more~" based on the student's answers.
    - Keeps the conversation concise, with a maximum of 100 characters. However, code examples can be up to 300 characters.
    - Multilingual support available. Korean by default, with options for English and Japanese.
    """,
    "ja": """
    コーディングチューターは、高校生の情報科目でアルゴリズムとプログラミングの実習をサポートするチューターボットです。
    - チューターは「こんにちは~」と挨拶からレッスンを始めます。
    - 生徒の学習レベルや進捗に応じて、個別のフィードバックや学習計画を提供します。
    - 選択したセクションの内容を要約し、レッスンを開始します。
    - 各セクションの概念を丁寧に説明し、理解を深めるための演習問題を提供します。
    - 生徒の回答に対して「よくできました！」、「もう少し考えてみましょう~」など、前向きなフィードバックを行います。
    - 会話は簡潔に、最大100文字以内で進行します。ただし、コード例は300文字以内で記述できます。
    - 多言語対応可能。デフォルトは韓国語、オプションで英語と日本語を選択できます。
    """
}

ui_texts = {
    "ko": {
        "title": "파이썬 튜터 봇",
        "user_id": "학생 ID",
        "start_button": "학습 시작",
        "submit_button": "질문 제출",
        "execute_button": "코드 실행",
        "code_input": "코드 입력",
        "code_output": "실행 결과",
        "lang_selector": "언어 선택",
        "change_lang_button": "언어 변경",
        "tutor_response": "튜터의 대답",
        "user_input": "학생의 질문"
    },
    "en": {
        "title": "Python Tutor Bot",
        "user_id": "Student ID",
        "start_button": "Start Learning",
        "submit_button": "Submit Question",
        "execute_button": "Run Code",
        "code_input": "Code Input",
        "code_output": "Execution Result",
        "lang_selector": "Language Selection",
        "change_lang_button": "Change Language",
        "tutor_response": "Tutor's Answer",
        "user_input": "Student's Question"
    },
    "ja": {
        "title": "パイソンチューターボット",
        "user_id": "学生 ID",
        "start_button": "学習を開始",
        "submit_button": "質問を提出",
        "execute_button": "コードを実行",
        "code_input": "コード入力",
        "code_output": "実行結果",
        "lang_selector": "言語選択",
        "change_lang_button": "言語を変更",
        "tutor_response": "チューターの答え",
        "user_input": "学生の質問"
    }
}

In [ ]:
# 채팅 함수
def chat_with_tutor(user_input, chat_history, user_profiles, user_id, lang="ko"):
    try:
        if user_id not in user_profiles:
            user_profiles[user_id] = {"progress": [], "lang": lang}

        messages = [{"role": "system", "content": system_messages[lang]}] + chat_history

        if not chat_history:
            messages.append({"role": "user", "content": "안녕하세요. 파이썬 기초에 대해서 학습을 지도해줘요"})

        messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model="gpt-4o-mini-2024-07-18",  # 일반적으로 사용 가능한 모델로 변경
            messages=[
                {"role": "system", "content": system_messages},
                {"role": "user", "content": "안녕하세요, 파이썬 기초에 대해서 학습을 지도해줘요"},
                {"role": "assistant", "content": "반갑습니다. 파이썬 기초에 학습해보도록 할게요. 먼저 파이썬의 기본 문법과 개념들을 간단히 요약해드립니다."},
                {"role": "user", "content": "변수와 자료형에 대해 학습해줘요"},
            ],
            temperature=0.5,
            max_tokens=300,
            top_p=1,
            user=user_id
        )

        response_msg = response.choices[0].message.content
        chat_history.append({"role": "assistant", "content": response_msg})
        user_profiles[user_id]["progress"].append(user_input)  # 학습 진도 기록

        return response_msg, chat_history, user_profiles

    except client.error.APIError as e:
        return f"Error: {e}", chat_history, user_profiles

    except Exception as e:
        return f"Unexpected error: {str(e)}", chat_history, user_profiles

In [ ]:
# 학습 시작 시 기본 설정
def start_chat(user_id, user_profiles):
    if user_id not in user_profiles:
        user_profiles[user_id] = {"progress": [], "lang": "ko"}
    greeting = "반가워요, 파이썬 기초를 학습해보도록 할게요. 먼저 파이썬의 기본문법과 개념들을 간단히 요약해드릴게요."
    return greeting, [], user_profiles

# 코드 실행 함수 (추가된 기능)
def execute_code(code):
    try:
        exec_globals = {}
        exec(code, exec_globals)
        return exec_globals
    except Exception as e:
        return str(e)

# 다국어 지원을 위한 언어 변경 기능
def change_language(lang, user_id, user_profiles):
    if user_id in user_profiles:
        user_profiles[user_id]["lang"] = lang
    return f"언어가 {lang}로 변경되었습니다.", user_profiles

# UI 업데이트 함수
def update_ui(lang, user_profiles, user_id):
    ui = ui_texts[lang]
    return (
        ui["title"], ui["user_id"], ui["start_button"], ui["submit_button"],
        ui["execute_button"], ui["code_input"], ui["code_output"],
        ui["lang_selector"], ui["change_lang_button"], ui["tutor_response"],
        ui["user_input"]
    ), user_profiles

In [ ]:
# UI 구성
with gr.Blocks() as demo:
    lang_selector = gr.Dropdown(choices=["ko", "en", "ja"], value="ko", label="언어 선택")

    ui_text = gr.State(ui_texts["ko"])
    user_profiles = gr.State({})
    user_id = gr.Textbox(label=ui_text.value["user_id"], value="default_user", interactive=True)
    chat_history = gr.State([])

    title = gr.Markdown(ui_text.value["title"])

    with gr.Row():
        start_button = gr.Button(ui_text.value["start_button"])
        tutor_response = gr.Textbox(label=ui_text.value["tutor_response"], interactive=False)
        user_input = gr.Textbox(label=ui_text.value["user_input"])
        submit_button = gr.Button(ui_text.value["submit_button"])

    with gr.Row():
        code_input = gr.Textbox(label=ui_text.value["code_input"])
        execute_button = gr.Button(ui_text.value["execute_button"])
        code_output = gr.Textbox(label=ui_text.value["code_output"])

    with gr.Row():
        change_lang_button = gr.Button(ui_text.value["change_lang_button"])

    # 언어 변경에 따른 UI 업데이트
    lang_selector.change(update_ui, inputs=[lang_selector, user_profiles, user_id], outputs=[
                         title, user_id, start_button, submit_button, execute_button, code_input, code_output, lang_selector, change_lang_button, tutor_response, user_input])

    # 학습 시작 버튼 클릭 시 호출
    start_button.click(start_chat, inputs=[user_id, user_profiles], outputs=[tutor_response, chat_history, user_profiles])

    # 질문 제출 버튼 클릭 시 호출
    submit_button.click(chat_with_tutor, inputs=[user_input, chat_history, user_profiles, user_id, lang_selector], outputs=[tutor_response, chat_history, user_profiles])

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://70c7d9e6940d1056f5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "<ipython-input-53-da9aace51ca6>", line 14, in chat_with_tutor
    response = client.chat.completions.create(
  File "/usr/local/lib/python3.10/dist-packages/openai/_utils/_utils.py", line 274, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/openai/resources/chat/completions.py", line 668, in create
    return self._post(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1260, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 937, in request
    return self._request(
  File "/usr/local/lib/python3.10/dist-packages/openai/_base_client.py", line 1041, in _request
    raise self._make_status_error_from_response(err.response) from None
openai.BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'messages[0].content[0]': expected

Task1_0826. input 함수가 처리되도록 코드를 수정해서 실행해 보세요.Task1_0826. input 함수가 처리되도록 코드를 수정해서 실행해 보세요.

In [ ]:
import gradio as gr
from openai import OpenAI
import json, os, sys, io

# OpenAI API 설정
client = OpenAI(api_key="")

# 시스템 메시지 설정
system_msg = """
코딩튜터는 고등학생 정보과목에서 알고리즘과 프로그래밍을 학생들이 실습할 수 있게 도와주는 튜터봇입니다.
- 튜터는 "반가와요~ "라고 인사말로 학습을 시작합니다.
- 학생이 선택한 섹션으로 학습을 시작하고 먼저 간단하게 섹션 학습내용 요약을 제공합니다.
- 각 섹션별 학습 항목에 대해서 하나씩 개념을 명확하고 친절하게 설명하고 연습문제를 추가로 제공해서 각 개념을 이해하여 활용할 수 있도록 연습 기회를 제공합니다.
- 학생의 답변에 대해서 "잘했어요!", "조금 더 생각해봐요~" 와 같은 긍정적인 피드백을 제공합니다.
- 대화는 간결하게 유지하며, 최대 100자 이내로 진행합니다. 단 코드는 300자 이내로 작성합니다.
- 튜터는 학생들이 자기주도 학습을 할 수 있도록 개인화 된 학습을 제공하고 모든 질의 응답을 한국어로 진행합니다.
"""

# 함수 정의
functions = [
    {
        "name": "answer_question",
        "description": "학생의 질문에 대한 답변을 제공합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "answer": {
                    "type": "string",
                    "description": "질문에 대한 텍스트 답변"
                }
            },
            "required": ["answer"]
        }
    },
    {
        "name": "execute_code",
        "description": "학생이 제출한 코드를 실행하고 결과를 반환합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "학생이 작성한 파이썬 코드"
                }
            },
            "required": ["code"]
        }
    },
    {
        "name": "handle_request",
        "description": "학생의 새로운 요청을 처리합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "request": {
                    "type": "string",
                    "description": "학생의 새로운 요청"
                }
            },
            "required": ["request"]
        }
    }
]

# 함수 호출 시뮬레이션
def answer_question(answer):
    return {"answer": answer}

# 코드 실행 함수 정의
def execute_code(code, user_input_value):
    # 여기서 코드를 실제로 실행하거나 안전하게 평가하는 로직을 추가할 수 있습니다.
    try:
        # 사용자로부터 입력을 받는 input() 함수 대체 (정규 표현식 사용)
        if user_input_value:
            # 정규 표현식으로 input()함수 전체를 대체(괄호포함)
            code = re.sub(r'input\((.*?)\)', f'"{user_input_value}"', code)
        # 디버깅을 위한 출력 : 대체된 코드 확인
        print(f"대체코드:\n{code}")

        # 코드 실행 환경설정
        exec_globals = {}
        exec_output = []

        # print 출력을 수집하기 위한 함수 재정의
        def custom_print(*args):
            exec_output.append(" ".join(map(str, args)))

        # print함수를 customer_print로 대체 (출력값 수집)
        exec_globals['print'] = custom_print

        # 코드 실행
        exec(code, exec_globals)

        # 실행 결과 반환 (수집된 출력)
        exec_output_str = '\n'.join(exec_output) if exec_output else "코드 실행 성공!"
        return exec_output_str
    except Exception as e:
        # 오류 발생시 반환
        return f"코드 실행 오류: {str(e)}"

# input() 함수가 있는지 확인하는 함수
def check_for_input_function(code):
    # 정규 표현식으로 input() 함수가 포함된 코드 감지
    return bool(re.search(r'input\((.*?)\)', code))

# gradio 인터페이스 정의
with gr.Blocks() as demo:
    # 코드 입력 및 input() 값을 받는 텍스트 박스
    code_input = gr.Textbox(label="코드 입력", placeholder="input()을 포함한 코드를 입력하세요", lines=10)

    # input() 값 입력창, 기본적으로 숨김
    user_input_value = gr.Textbox(label="input()값 입력", placeholder="input() 값", visible=False)

    # 결과 출력 텍스트박스
    code_output = gr.Textbox(label="코드 실행 결과", placeholder="여기에 실행결과가 표시")

    # 코드 실행 버튼
    code_run_button = gr.Button("코드 실행")

    # 코드에서 input() 함수가 있는지 확인하여 입력창 표시
    def handle_code_execution(code):
        # input() 함수가 있는지 확인 (정규표현식 사용)
        if check_for_input_function(code):
            return gr.update(visible=True), "" # input 창을 보이게 하고, 실행 결과는 아직 없음
        else:
            output = execute_code(code, None)
            return gr.update(visuble=False), output # input 창을 숨기고, 코드 바로 실행

    # 최종 코드 실행 (input() 함수 값 입력 후)
    def execute_with_input(code, user_input_value):
        output = execute_code(code, user_input_value)
        return output

    # 버튼 클릭시 코드에서 input()을 감지해 창을 표시하거나 바로 실행
    code_run_button.click(handle_code_execution, inputs=[code_input], outputs=[user_input_value, code_output])

    # input() 값 입력 후 코드 실행
    user_input_value.submit(execute_with_input, inputs=[code_input, user_input_value], outputs=[code_output])

demo.launch(debug=True)